In [1]:
import nltk
import pickle
import string
import pandas as pd

from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tag import pos_tag
from random import shuffle

In [2]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

In [3]:
stemmer = PorterStemmer()
wnl = WordNetLemmatizer()
eng_stopwords =stopwords.words('english')

dataset = pd.read_csv('updated_dataset.csv')

In [4]:
def preprocessing(doc):
    words = word_tokenize((doc).lower())
    words = [wnl.lemmatize(word) for word in words]
    words = [stemmer.stem(word) for word in words]

    return {word: True for word in words if word not in eng_stopwords and word.isalpha() and word not in string.punctuation}

In [6]:
def trainModel():
    feature_sets = [(preprocessing(text), label) for text, label in zip(dataset['text'], dataset['label'])]
    shuffle(feature_sets)
    index = int(len(feature_sets) * 0.85)
    train_set, test_set = feature_sets[:index], feature_sets[index:]

    classifier = nltk.NaiveBayesClassifier.train(train_set)

    accuracy = nltk.classify.accuracy(classifier, test_set)
    print('Accuracy: ', accuracy)

    classifier.show_most_informative_features(5)

    file = open('model.pickle', 'wb')
    pickle.dump(classifier, file)
    file.close()

    return classifier

def readModel():
    try:
        file = open('model.pickle', 'rb')
        classifier = pickle.load(file)
        file.close()
        return classifier
    except:
        print("Model is not available")
        print("Training model...")
        return trainModel()


In [10]:
def writeReview():
    while True:
        review = input("Enter review[>=2 words]: ")
        words = review.split()

        if len(words) >= 2:
            print("Review added!")
            break
        print("Review should have atleast 2 words")
    return review

In [11]:
def analyzeReview(review, classifier):
    if(len(review) == 0):
        print("No review found")
        return
    words = word_tokenize(review.lower())
    words = FreqDist([word for word in words if word.isalpha() and word not in string.punctuation])

    tag = pos_tag(words)
    print("POS tag:")
    for i, word in enumerate(tag):
        print(f"{i + 1}. {word[0]}: {word[1]}")
    
    for word in words:
        print(f"Word: {word}")
        print("=============================")

        synsets = wordnet.synsets(word)
        synonyms = [] # or use set() to avoid duplicates
        antonyms = []

        for synset in synsets:
            for lemma in synset.lemmas():
                synonyms.append(lemma.name())
                for antonym in lemma.antonyms():
                    antonyms.append(antonym.name())

        if(len(synonyms) == 0):
            print("No synonyms found")
        else:
            print("Synonyms:")
            for syn in synonyms:
                print(syn)
        
        print("")

        if(len(antonyms) == 0):
            print("No antonyms found")
        else:
            print("Antonyms:")
            for ant in antonyms:
                print(ant)
    clean_review = [word for worn in word_tokenize(review.lower()) if word.isalpha() and word not in string.punctuation and word not in eng_stopwords]

    result = classifier.classify(FreqDist(clean_review))

    print(f"Review is {result}")

    return result


In [13]:
classifier = readModel()

review = ""
sentiment = ""

while True:
    print("Review Sentiment Analysis")
    print("Your Review: ", "No Review" if len(review) == 0 else review)
    print("Sentiment: ", "No Sentiment" if len(sentiment) == 0 else sentiment)
    print("1. Write Review")
    print("2. Analyze Review")
    print("3. Exit")
    choice = int(input("Enter choice: "))
    if choice == 1:
        review = writeReview()
    elif choice == 2:
        sentiment = analyzeReview(review, classifier)
    elif choice == 3:
        break
    else:
        print("Invalid choice")
        continue
    

Review Sentiment Analysis
Your Review:  No Review
Sentiment:  No Sentiment
1. Write Review
2. Analyze Review
3. Exit
Review added!
Review Sentiment Analysis
Your Review:  Very good nice
Sentiment:  No Sentiment
1. Write Review
2. Analyze Review
3. Exit
POS tag:
1. very: RB
2. good: JJ
3. nice: NN
Word: very
Synonyms:
very
identical
selfsame
very
very
really
real
rattling
very

No antonyms found
Word: good
Synonyms:
good
good
goodness
good
goodness
commodity
trade_good
good
good
full
good
good
estimable
good
honorable
respectable
beneficial
good
good
good
just
upright
adept
expert
good
practiced
proficient
skillful
skilful
good
dear
good
near
dependable
good
safe
secure
good
right
ripe
good
well
effective
good
in_effect
in_force
good
good
serious
good
sound
good
salutary
good
honest
good
undecomposed
unspoiled
unspoilt
good
well
good
thoroughly
soundly
good

Antonyms:
evil
evilness
bad
badness
bad
evil
ill
Word: nice
Synonyms:
Nice
nice
decent
nice
nice
skillful
dainty
nice
overnice
pri